VERİ TEMİZLEME

In [1]:
#metinlerde bulunan fazla boşlukları kaldır
text = "hello      world        2035"
cleaned_text = " ".join(text.split())
cleaned_text

'hello world 2035'

In [2]:
#noktalama işaretlerini kaldır
import string

text1 = "hello!! world , 2035,"

cleaned_text1 = text1.translate(str.maketrans("","",string.punctuation))
cleaned_text1

'hello world  2035'

In [3]:
#özel karakterleri kadlr
import re

text2 = "hello, world' 2035%"
cleaned_text2= re.sub(r"[^A-Za-z0-9\s]","",text2)
cleaned_text2

'hello world 2035'

In [4]:
#yazım hatalarını düzelt
from textblob import TextBlob #metin analizlerinde kullanılan bir kütüphane
text3 = "hellio, wırld 2035"
cleaned_text3 = TextBlob(text3).correct() #correct: yazım hatalarını düzeltir
cleaned_text3 

TextBlob("hello, world 2035")

In [5]:
#html ya da url etiketlerini kaldır
from bs4 import BeautifulSoup

html_text = "<div>Hello world 2035<div>"
cleaned_text4 = BeautifulSoup(html_text,"html.parser").get_text()
cleaned_text4

'Hello world 2035'

TOKENİZATİON

In [6]:
import nltk #natural language toolkit
nltk.download("punkt") #metki kelime ve cümle bazında tokenlara ayırmak için gerekli

text5=  "Hello WOrld, 22035! hi , hello ..." 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kezer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
#kelime tokenizasyonu : word_tokenize: metni kelimelere ayrıır, noktalama işaretleri ve boşluklar ayrı birer token olarak elde edilecektir.
word_tokens = nltk.word_tokenize(text5)
word_tokens

['Hello', 'WOrld', ',', '22035', '!', 'hi', ',', 'hello', '...']

In [8]:
#cümle tokenizasyonu: sent_tokenize: metni cümlelere ayırır. her bir cümle birer token olur
sentence_tokens =nltk.sent_tokenize(text5)
sentence_tokens

['Hello WOrld, 22035!', 'hi , hello ...']

STEMMING & LEMMIZATION

In [9]:
import nltk
nltk.download("wordnet") #wordnet : lemmization işlemi için gerekli veritabanı 

from nltk.stem import PorterStemmer #stemming için fonksiyon

#porter stemmer nesnesini oluştur
stemmer =PorterStemmer()

words = ["runner","running" ,"ran" , "runs", "better", "go", "went"]

#kelimelerin stemlerini buluyoruzbunu yaparkende porter stemmerın stem() fonksiyonunu kullanıyoruz
stems = [stemmer.stem(w) for w in words]
stems


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kezer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['runner', 'run', 'ran', 'run', 'better', 'go', 'went']

In [10]:
#lemmezation
from nltk.stem import WordNetLemmatizer
lemmetizer = WordNetLemmatizer()

words = ["runner","running" ,"ran" , "runs", "better", "go", "went"]

#kelimelerin stemlerini buluyoruzbunu yaparkende porter stemmerın stem() fonksiyonunu kullanıyoruz
lemma = [lemmetizer.lemmatize(w, pos="v") for w in words]
lemma

['runner', 'run', 'run', 'run', 'better', 'go', 'go']

STOPWORDS

In [11]:
from nltk.corpus import stopwords
nltk.download("stopwords")  # farkli dillerde en çok kullanılan stop words içeren kütüphaneler

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kezer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#ingilizce stop words analizi
stop_words_eng = set(stopwords.words("english"))

text = "There are some ezamples of handling stop words from some texts"
text_list = text.split()

#eğer bir kelime stopword listesinde yoksa bu kelimeyi filtrelenmiş listesine ekliyoruz 
filtered_words = [word for word in text_list if word.lower() not in stop_words_eng]
filtered_words

['ezamples', 'handling', 'stop', 'words', 'texts']

In [13]:
#türkçe stop words analizi
stop_words_tr = set(stopwords.words("turkish"))

text = "merhaba arkadaşlar bir çok güzel falan filan"
text_list = text.split()

#eğer bir kelime stopword listesinde yoksa bu kelimeyi filtrelenmiş listesine ekliyoruz 
filtered_words = [word for word in text_list if word.lower() not in stop_words_tr]
filtered_words

['merhaba', 'arkadaşlar', 'bir', 'güzel', 'falan', 'filan']

In [60]:
#kütühanesiz stopwords listesi çıkarma
#stop words listesi hazırlama

tr_stopwords = ["için", "bu", "ile", "mu", "mi"]

text = "bu bir denemedir. mi acaba?"
text_list = text.split() 


#eğer bir kelime stopword listesinde yoksa bu kelimeyi filtrelenmiş listesine ekliyoruz 
filtered_words = [word for word in text_list if word not in tr_stopwords]
filtered_words

['bir', 'denemedir', '.', 'acaba?']

BAG OF WORDS(BoW)

In [15]:
#count vectorizer içeri aktar
from sklearn.feature_extraction.text import CountVectorizer

# veri seti oluştur
documents = [ "kedi bahçede", "kedi evde"]

#vectorizer tanımla
vectorizer =CountVectorizer()

#metni sayısal vektörlere çevir
X = vectorizer.fit_transform(documents)

#kelime kümesini oluşturma
feature_names = vectorizer.get_feature_names_out(X) 
print(feature_names)

#ve vektör temsili
vector_temsili = X.toarray()
vector_temsili

['bahçede' 'evde' 'kedi']


array([[1, 0, 1],
       [0, 1, 1]], dtype=int64)

BOW IMDB DATASET

In [17]:
#import libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import re
from collections import Counter

df= pd.read_csv("IMDB Dataset.csv")

In [23]:
#metin verilerini alalım
documents = df["review"]
labels = df["sentiment"] #pozitif veya negatif  

In [20]:
# metin temzileme
def clean_text(text):

    #büyük küçük harf çevrimi
    text= text.lower()

    #rakamları temizleme
    text = re.sub(r"\d+","", text)

    #özel karakterlerin kaldırılmaıs
    text = re.sub(r"[^\w\s]","",text)

    #kısa kelimelerin temizlenmesi
    text = " ".join([word for word in text.split() if len(word)> 2])

    return text 

In [48]:
#metinleri temizle
cleaned_doc = [clean_text(row) for row in documents]
cleaned_doc

['one the other reviewers has mentioned that after watching just episode youll hooked they are right this exactly what happened with mebr the first thing that struck about was its brutality and unflinching scenes violence which set right from the word trust this not show for the faint hearted timid this show pulls punches with regards drugs sex violence its hardcore the classic use the wordbr called that the nickname given the oswald maximum security state penitentary focuses mainly emerald city experimental section the prison where all the cells have glass fronts and face inwards privacy not high the agenda city home manyaryans muslims gangstas latinos christians italians irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awaybr would say the main appeal the show due the fact that goes where other shows wouldnt dare forget pretty pictures painted for mainstream audiences forget charm forget romanceoz doesnt mess around the first episode ever saw s

BoW

In [61]:
from nltk.corpus import stopwords
nltk.download("stopwords")  # farkli dillerde en çok kullanılan stop words içeren kütüphaneler

#türkçe stop words analizi
stop_words_eng = set(stopwords.words("english"))

text_list = [i.split() for i in cleaned_doc]

# Tüm kelimeleri bir listeye düzleştirme
flattened_text_list = [word for sublist in text_list for word in sublist]

#eğer bir kelime stopword listesinde yoksa bu kelimeyi filtrelenmiş listesine ekliyoruz 
filtered_words = [word for word in flattened_text_list if word.lower() not in stop_words_eng]
filtered_words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kezer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['one',
 'reviewers',
 'mentioned',
 'watching',
 'episode',
 'youll',
 'hooked',
 'right',
 'exactly',
 'happened',
 'mebr',
 'first',
 'thing',
 'struck',
 'brutality',
 'unflinching',
 'scenes',
 'violence',
 'set',
 'right',
 'word',
 'trust',
 'show',
 'faint',
 'hearted',
 'timid',
 'show',
 'pulls',
 'punches',
 'regards',
 'drugs',
 'sex',
 'violence',
 'hardcore',
 'classic',
 'use',
 'wordbr',
 'called',
 'nickname',
 'given',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 'focuses',
 'mainly',
 'emerald',
 'city',
 'experimental',
 'section',
 'prison',
 'cells',
 'glass',
 'fronts',
 'face',
 'inwards',
 'privacy',
 'high',
 'agenda',
 'city',
 'home',
 'manyaryans',
 'muslims',
 'gangstas',
 'latinos',
 'christians',
 'italians',
 'irish',
 'moreso',
 'scuffles',
 'death',
 'stares',
 'dodgy',
 'dealings',
 'shady',
 'agreements',
 'never',
 'far',
 'awaybr',
 'would',
 'say',
 'main',
 'appeal',
 'show',
 'due',
 'fact',
 'goes',
 'shows',
 'wouldnt',
 'da

In [62]:
#vectorizer tanımla
vectorizer= CountVectorizer()

#metin sayısal hale getirilir
X = vectorizer.fit_transform(filtered_words[:75])

#kelime kümesini göster
feature_names1 = vectorizer.get_feature_names_out()

#vektör temsiliv
vektör_temsili = X.toarray()
vektör_temsili

df_bow = pd.DataFrame(vektör_temsili, columns=feature_names1)
df_bow

,agenda,brutality,called,cells,christians,city,classic,dealings,death,dodgy,...,thing,timid,trust,unflinching,use,violence,watching,word,wordbr,youll
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
72,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [63]:
#kelime frekanslarınıj göster
word_counter = X.sum(axis=0).A1
word_freq =dict(zip(feature_names1,word_counter))
word_freq

{'agenda': 1,
 'brutality': 1,
 'called': 1,
 'cells': 1,
 'christians': 1,
 'city': 2,
 'classic': 1,
 'dealings': 1,
 'death': 1,
 'dodgy': 1,
 'drugs': 1,
 'emerald': 1,
 'episode': 1,
 'exactly': 1,
 'experimental': 1,
 'face': 1,
 'faint': 1,
 'first': 1,
 'focuses': 1,
 'fronts': 1,
 'gangstas': 1,
 'given': 1,
 'glass': 1,
 'happened': 1,
 'hardcore': 1,
 'hearted': 1,
 'high': 1,
 'home': 1,
 'hooked': 1,
 'inwards': 1,
 'irish': 1,
 'italians': 1,
 'latinos': 1,
 'mainly': 1,
 'manyaryans': 1,
 'maximum': 1,
 'mebr': 1,
 'mentioned': 1,
 'moreso': 1,
 'muslims': 1,
 'nickname': 1,
 'one': 1,
 'oswald': 1,
 'penitentary': 1,
 'prison': 1,
 'privacy': 1,
 'pulls': 1,
 'punches': 1,
 'regards': 1,
 'reviewers': 1,
 'right': 2,
 'scenes': 1,
 'scuffles': 1,
 'section': 1,
 'security': 1,
 'set': 1,
 'sex': 1,
 'show': 2,
 'stares': 1,
 'state': 1,
 'struck': 1,
 'thing': 1,
 'timid': 1,
 'trust': 1,
 'unflinching': 1,
 'use': 1,
 'violence': 2,
 'watching': 1,
 'word': 1,
 'wordbr

In [64]:
#ilk 5 kelimey yazdır
most_common_5_words = Counter(word_freq).most_common(5)
most_common_5_words

[('city', 2), ('right', 2), ('show', 2), ('violence', 2), ('agenda', 1)]